This project is inspired by  standford project report. Both pdf and the github repository of the GAN models for this project are added as follows for the Reference:

* https://github.com/goldhuang/SRGAN-PyTorch

* http://cs230.stanford.edu/projects_fall_2018/reports/12445558.pdf


Also, to calculate the Multi-Scale Structural Similarity (SSIM) index, we used the following repo as our reference

* https://github.com/jorge-pessoa/pytorch-msssim



To compute QILV, we converted codes gained from mathlab function by following address to the Python
* https://www.mathworks.com/matlabcentral/fileexchange/36950-quality-index-based-on-local-variance-qilv

* source : http://poseidon.tel.uva.es/~santi/personal/embc06.pdf

The folowoing labriries are recommended to be installed to support memory footprint libraries/code for using the GPU 


* !pip install gputil
* !pip install psutil
* !pip install humanize


In [ ]:
# the ssim calculation libraries 
import math
from math import exp
from numpy.linalg import LinAlgError

# import liberaries for Data utils 
import os
from os import listdir
from os.path import join
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms
from torchvision.transforms import Compose, RandomCrop, CenterCrop, Resize, ToTensor, ToPILImage, Normalize

# import liberaries for model classes 
import torch
from torchvision import models, datasets
from torch import nn,optim
from torch.nn.utils import weight_norm
from torch.nn import SyncBatchNorm
from collections import OrderedDict
import torch.autograd as autograd
import torch.nn.functional as F
from torch.autograd import Variable



# import liberaries for train and validation the SRGAN model 

from math import log10
import pandas as pd
import torch.optim as optim
import torch.utils.data 
from torch.utils.data import DataLoader
import torchvision.utils as utils
from tqdm import tqdm


# import the necessary packages
import numpy as np
import random
import time
from time import gmtime, strftime

start_localtime = strftime("%Y-%m-%d %H:%M:%S", gmtime())


# import the liberaries to support memory footprint libraries/code for using the GPU
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]





In [ ]:
# psutil is a cross-platform library for retrieving information on running processes and system utilization (CPU, memory, disks, network)
# humanize offers various utilities which can be used on the numbers to make the numbers easily readable for the humans. 
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm() 


In [ ]:
# This project could run in Colab (if running from the computer, only importing the calsses is necesseray)
from google.colab import drive
drive.mount('/content/drive')
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Replace id# with an appropriate Key gained from the colab drive
quality_assessment= drive.CreateFile({'id': 'id#'})
quality_assessment.GetContentFile('quality.py')

models= drive.CreateFile({'id': 'id#'})
models.GetContentFile('models.py')

dataloader= drive.CreateFile({'id': 'id#'})
dataloader.GetContentFile('dataloader.py')

getparam= drive.CreateFile({'id': 'id#'})
getparam.GetContentFile('getparam.py')


# After authentication, we can import our file liberiries as follow.
from quality import *
from models import *
from dataloader import *
from getparam import *

In [ ]:
# initialize the parameters 

#####################################
crop_size =256
upscale_factor=4  # the upscale_factor can be changed based on the paper
num_batch_size= 1
model='wa_srgan_WN'
num_epoch=500
chech_point=-1
n_epoch_pretrain = 2
#################################

learning_rate=1e-3
lr_decay_steps=200
lr_decay_gamma=0.5
seed=1
rgb_mean=[0.4488,0.4371,0.4040]
rgb_range=(0.0,255.0)
torch.backends.cudnn.benchmark=True

# if there is GPU, train from GPU
device= ('cuda:0' if torch.cuda.is_available() else 'cpu')

random.seed(seed)
torch.manual_seed(seed)

str_root='/content/drive' # if running from Colab

# the folders that will be used during the training
train_dataset_dir= str_root+'data/Data/train_HR'
val_dataset_dir= str_root+'data/Data/valid_HR'
epoch_path_valing= str_root+'data/epoch/'
str_statistics= str_root+'data/statistics/'
str_results= str_root+'data/training_results/'



In [ ]:

# load the images in the train loaders
train_set= TrainDatasetFromFolder(train_dataset_dir, crop_size=crop_size, upscale_factor=upscale_factor)
val_set=ValDatasetFromFolder(val_dataset_dir, upscale_factor=upscale_factor,crop_size=crop_size)
trian_loader=DataLoader(train_set, batch_size=num_batch_size, shuffle=True, num_workers=256)
val_loader=DataLoader(val_set, batch_size=1, shuffle=False, num_workers=256)


In [ ]:
# Vgg 19 has already pre trained. The weights are loaded to calculate the loss
state_dict_vgg=torch.load(str_root+"vgg19.pth")
# set the model, loss fucnction(critirion) and optimization function
netG=Generator_Self_Attention_WGAN_WDSR_WN(upscale_factor)
netD=Discriminator_Self_Attention_WGAN_WN()
critirion=nn.MSELoss()
generator_critirion=GeneratorLoss(state_dict_vgg,model)
optimizerG=optim.Adam(netG.parameters(),lr=1e-4)
optimizerD=optim.Adam(netD.parameters(),lr=1e-4)

# print the G and D models 
print(netG)
print(netD)

Generator_Self_Attention_WGAN_WDSR_WN(
  (block1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
    (1): PReLU(num_parameters=1)
  )
  (Resblock_wdsr): Sequential(
    (0): ResBlock_wdsr_WN(
      (module): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(256, 51, kernel_size=(1, 1), stride=(1, 1))
        (3): Conv2d(51, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (1): ResBlock_wdsr_WN(
      (module): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(256, 51, kernel_size=(1, 1), stride=(1, 1))
        (3): Conv2d(51, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (2): ResBlock_wdsr_WN(
      (module): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): ReLU(inplace=True)
        (2): Conv2d(256, 51,

In [ ]:

# Trianing, and validating the WA-SRGAN model

print('%s model training is about to start ... you may find the results in the data/ folder' % (model))
print('Start time : %s '%(start_localtime))
end_localtime = strftime("%Y-%m-%d %H:%M:%S", gmtime())
#get the parameters name of the top and the bottom of the netD and netG
netG_param_top,netG_param_bottom=get_param_names(netG)
netD_param_top,netD_param_bottom=get_param_names(netD)



# if torch.cuda.is_available():
netD.to(device)
netG.to(device)
generator_critirion.to(device)
if chech_point==-1:
  # optimizerG = optim.Adam(netG.parameters())
  for epoch in range(1, n_epoch_pretrain + 1):
    pre_train_bar = tqdm(trian_loader)
    netG.train()
    pre_trained_running_results = {'g_loss': 0, 'pre_training_num':0}
    for target, data in pre_train_bar:
      real_img=target.to(device)
      data=data.to(device)
      fake_img=netG(data)

      # Train G
      netG.zero_grad()
      image_loss = critirion(fake_img, real_img)
      pre_trained_running_results['g_loss'] += image_loss
      pre_trained_running_results['pre_training_num'] += 1
      image_loss.backward()
      optimizerG.step()

      # Print information by tqdm
      pre_train_bar.set_description(desc='[%d/%d] Loss_G: %.4f' % (epoch, n_epoch_pretrain, image_loss))



results={'d_loss':[], 'g_loss':[],'d_score':[], 'g_score':[],'psnr':[],'ssim':[],'psnr_lr':[],'ssim_lr':[],'dtg':[], 'dbg':[], 'gtg':[], 'gbg':[],'msssim':[],'qilv':[],'mse':[] ,'msssim_lr':[],'qilv_lr':[],'mse_lr':[] }
# pre train the generator beforehand 

for epoch in range(1, num_epoch+1):
  train_bar=tqdm(trian_loader)
  running_results={'batch_sizes':0, 'd_loss':0, 'g_loss':0, 'g_score':0, 'd_score':0, 'training_num':0,'dtg':0, 'dbg':0, 'gtg':0, 'gbg':0 }
  netG.train()
  netD.train()
  for target, data  in train_bar:
    batch_size=data.size(0)
    running_results['batch_sizes']+=batch_size
    running_results['training_num']+=1
    ########################################
    #update D-network min(D(G(z))-D(x)+ 10* (second gradient on the (norm of( gradient of X- gradient of Xhat / x-xhat))-1).mean
    # {mean is because we have batch size}
    #########################################
    real_img=Variable(target)
    z=Variable(data)
    # if torch.cuda.is_available():
    real_img=real_img.to(device)
    z=z.to(device)
    fake_img=netG(z)
    netD.zero_grad()
    
    ###########################
    # 1- train with real
    #############################
    real_out=torch.mean(netD(real_img))
    # real_out.backward()
    #############################
    # 2-  # train with fake
    ##############################
    fake_out=torch.mean(netD(fake_img))

    #################################
    gradient_penalty = calc_gradient_penalty(netD, real_img, fake_img, device)
    d_loss = fake_out - real_out + 10*gradient_penalty
    running_results['d_loss']+=torch.sigmoid(d_loss).item()
    d_loss.backward(retain_graph=True)
    # Wasserstein_D = fake_out-real_out
    optimizerD.step()
    
    dtg, dbg = get_grads_D(netD,netD_param_top,netD_param_bottom)
    ###############################
    #update G-network min(1-D(G(z))+perception_loss+Image_loss+TV_loss)
    netG.zero_grad()
    g_loss=generator_critirion(fake_out.item(), fake_img, real_img)
    running_results['g_loss']+=g_loss.item()
    g_loss.backward()
    fake_img=netG(z)
    fake_out=torch.mean(netD(fake_img))
    optimizerG.step()
    gtg, gbg = get_grads_G(netG,netG_param_top,netG_param_bottom)
    #loss for current batch before optimization
    
    running_results['g_score']+=torch.sigmoid(fake_out).item()
    running_results['d_score']+=torch.sigmoid(real_out).item()
    running_results['dtg']+=dtg.item()
    running_results['dbg']+=dbg.item()
    running_results['gtg']+=gtg.item()
    running_results['gbg']+=gbg.item()
    train_bar.set_description(desc='[%d/%d] Loss_d: %.4f, Loss_g: %.4f, D(x): %.4f, D(G(z)) : %.4f, D grads:(%f, %f) G grads:(%f, %f)' % 
                              (epoch,num_epoch, 
                               running_results['d_loss']/running_results['training_num'] ,
                               running_results['g_loss']/running_results['training_num'] ,
                               running_results['d_score']/running_results['training_num'] ,
                               running_results['g_score']/running_results['training_num'] , 
                               running_results['dtg']/running_results['training_num'] ,
                               running_results['dbg']/running_results['training_num'] ,
                               running_results['gtg']/running_results['training_num'] ,
                               running_results['gbg']/running_results['training_num'] 
        
                       
                            ))
  netG.eval()
  out_path= str_results
  # if not os.out_path.exists():
  #   os.makedirs(out_path)
  with torch.no_grad():
    val_bar=tqdm(val_loader)
    valing_results={'mse':0, 'ssims': 0, 'psnr':0, 'ssim':0,'batch_sizes':0 ,'valing_num':0,
                    'msssim':0,'qilv':0,'mse_lr':0, 'ssims_lr': 0, 'psnr_lr':0, 'ssim_lr':0,'msssim_lr':0,'qilv_lr':0,'max_value':0}
    val_images=[]
    for val_lr, val_hr_restore, val_hr in val_bar:
        batch_size=val_lr.size(0)
        valing_results['batch_sizes']+=batch_size
        valing_results['valing_num']+=1
        lr=val_lr
        hr=val_hr
        hr_bicubic=val_hr_restore
        lr=lr.to(device)
        hr=hr.to(device)
        hr_bicubic=hr_bicubic.to(device)
        sr=netG(lr)

        if model=='wdsr':
          sr_quantize=quantize(sr,rgb_range)
          mse = ((sr_quantize - hr) ** 2).data.mean()
          valing_results['mse'] += mse.item()
          psnr = 10 * log10(255.0/(valing_results['mse'] / valing_results['valing_num']))
        else:
          mse = ((sr - hr) ** 2).data.mean()
          valing_results['mse'] += mse.item()
          psnr = 10 * log10(1.0/(valing_results['mse'] / valing_results['valing_num']))

        mse_lr=((hr_bicubic - hr) ** 2).data.mean()
        valing_results['mse_lr'] += mse_lr.item()
        psnr_lr = 10 * log10(1/(valing_results['mse_lr'] / valing_results['valing_num']))
       
        batch_ssim=ssim(sr,hr).item()
        batch_ssim_lr=ssim(hr_bicubic,hr).item()
        valing_results['psnr']+=psnr
        valing_results['psnr_lr']+=psnr_lr
        valing_results['ssim']+=batch_ssim
        valing_results['ssim_lr']+=batch_ssim_lr
        valing_results['msssim']+=msssim(sr,hr).item()
        valing_results['msssim_lr']+=msssim(hr_bicubic,hr).item()
        valing_results['qilv']+=qilv(sr,hr).item()
        valing_results['qilv_lr']+=qilv(hr_bicubic,hr).item()

        val_bar.set_description(desc='[converting LR images to SR ones with %d upscale]  mse: %.4f db PSNR: %.4f db SSIM: %.4f db MSSSIM: %.4f db QILV: %.4f '% 
                                (upscale_factor,mse,psnr,batch_ssim,msssim(sr,hr).item(),qilv(sr,hr).item(),))
        val_images.extend([
                          display_transform(crop_size)(val_hr_restore.squeeze(0)),
                          display_transform(crop_size)(hr.data.cpu().squeeze(0)),
                          display_transform(crop_size)(sr.data.cpu().squeeze(0))
                          ])
    
    val_images =torch.stack(val_images)
    val_images=torch.chunk(val_images, val_images.size(0)//3)
    val_save_bar=tqdm(val_images, desc='[saving training results]')
    index=1
    for image in val_save_bar :
      image = utils.make_grid(image, nrow=3, padding=5)
      if index <=2:
        utils.save_image(image, out_path+model+'_epoch_%d_size_%d_upscale_%d_index_%d.png'%(epoch,crop_size,upscale_factor,index), padding=5)
      index+=1
    # for index in range(1,11):
    # image= next(iter(val_save_bar))
    # image = utils.make_grid(image, nrow=3, padding=5)
    # utils.save_image(image, out_path+'epoch_%d_index_%d.png'%(epoch, index), padding=5)
  #save model parameters 
  
  torch.save(netG.state_dict(), epoch_path_valing +model+'_netG_epoch_%d_%d_%d.pth'%(epoch, crop_size, upscale_factor))
  torch.save(netD.state_dict(), epoch_path_valing+model+'_netD_epoch_%d_%d_%d.pth'%(epoch, crop_size, upscale_factor))
  #save loss/score/psnr/ssim
  results['d_loss'].append(running_results['d_loss']/running_results['training_num'] )
  results['g_loss'].append(running_results['g_loss']/running_results['training_num'])
  results['d_score'].append(running_results['d_score']/running_results['training_num'] )
  results['g_score'].append(running_results['g_score']/running_results['training_num'])
  results['dtg'].append(running_results['dtg']/running_results['training_num'])
  results['dbg'].append(running_results['dbg']/running_results['training_num'])
  results['gtg'].append(running_results['gtg']/running_results['training_num'])
  results['gbg'].append(running_results['gbg']/running_results['training_num'])
  results['mse'].append(valing_results['mse']/valing_results['valing_num'])
  results['mse_lr'].append(valing_results['mse_lr']/valing_results['valing_num'])
  results['psnr'].append(valing_results['psnr']/valing_results['valing_num'])
  results['psnr_lr'].append(valing_results['psnr_lr']/valing_results['valing_num'])
  results['ssim'].append(valing_results['ssim']/valing_results['valing_num'])
  results['ssim_lr'].append(valing_results['ssim_lr']/valing_results['valing_num'])
  results['msssim'].append(valing_results['msssim']/valing_results['valing_num'])
  results['msssim_lr'].append(valing_results['msssim_lr']/valing_results['valing_num'])
  results['qilv'].append(valing_results['qilv']/valing_results['valing_num'])
  results['qilv_lr'].append(valing_results['qilv_lr']/valing_results['valing_num'])
 
  if epoch % 1 == 0 and epoch !=0:
    out_path= str_statistics
    data_frame=pd.DataFrame(
        data={
            'Loss_D':results['d_loss'],
            'Loss_G':results['g_loss'],
            'Score_D':results['d_score'],
            'Score_G':results['g_score'],
            'PSNR':results['psnr'],
            'PSNR_BICUBIC':results['psnr_lr'],
            'MSE':results['mse'],
            'MSE_BICUBIC':results['mse_lr'],
            'SSIM':results['ssim'],
            'SSIM_BICUBIC':results['ssim_lr'],
            'MSSSIM':results['msssim'],
            'MSSSIM_BICUBIC':results['msssim_lr'],
            'QILV':results['qilv'],
            'QILV_BICUBIC':results['qilv_lr'],
            'd_top_grad':results['dtg'],
            'd_bot_grad':results['dbg'],
            'g_top_grad':results['gtg'],
            'g_bot_grad':results['gbg'],
         
        },
        index=range(1,epoch+1)
    )
    data_frame.to_csv(out_path+model+'_size_'+ str(crop_size) +'_upscale_'+str(upscale_factor)+ '_eval_results.csv', index_label="Epoch")

end_localtime = strftime("%Y-%m-%d %H:%M:%S", gmtime())
print('End time : %s '%(end_localtime))


  0%|          | 0/2000 [00:00<?, ?it/s]

wa_srgan_WN model training is about to start ... you may find the results in the data/ folder
Start time : 2020-06-30 08:10:39 


[2/2] Loss_G: 0.0019: 100%|██████████| 2000/2000 [02:05<00:00, 15.91it/s]
[1/500] Loss_d: 0.1090, Loss_g: -0.0186, D(x): 0.9278, D(G(z)) : 0.8746, D grads:(0.709609, 0.338008) G grads:(0.000135, 0.000070): 100%|██████████| 2000/2000 [06:16<00:00,  5.32it/s]
[converting LR images to SR ones with 4 upscale]  mse: 0.0078 db PSNR: 20.8912 db SSIM: 0.7994 db MSSSIM: 0.9532 db QILV: 0.9033 : 100%|██████████| 301/301 [00:46<00:00,  6.43it/s]
[saving training results]: 100%|██████████| 301/301 [00:00<00:00, 729.11it/s]
[2/500] Loss_d: 0.0150, Loss_g: -0.0265, D(x): 0.9990, D(G(z)) : 0.9925, D grads:(0.725749, 0.357063) G grads:(0.000105, 0.000055):  45%|████▍     | 896/2000 [03:01<03:09,  5.84it/s]